In [1]:
#Import libraries
import numpy as np
import os as os
import pickle as pickle
from cymetric.pointgen.pointgen import PointGenerator
import tensorflow as tf
tfk = tf.keras
from cymetric.models.tfhelper import prepare_tf_basis, train_model

In [2]:
#Set up point generator
n_coords = 5
monomial_powers = 5
monomials = monomial_powers*np.eye(n_coords, dtype=np.int64)
coefficients = np.ones(n_coords)
kmoduli = np.ones(1)
ambient = np.array([n_coords-1])
#print(monomials,coefficients,kmoduli,ambient)
pg = PointGenerator(monomials, coefficients, kmoduli, ambient)

#Generate Points
n_pts = 1000
points = pg.generate_points(n_pts)
#print(f'CY condition satisfied?  -->  {np.all(pg.cy_condition(points) < 1e-10)}')


In [3]:
#Import the pre-trained CYMetric model
cymodel_filepath = '/Users/ed/Documents/Projects/G2/Flows/G2Metric/github/cy_models/test_model.keras'
cymetric_model = tf.keras.models.load_model(cymodel_filepath)
#cymetric_model.summary()

In [4]:
#Define the coordinate change functions
def CoordChange_C5R10(point, inverse=False):
    ## TO DO: vectorise
    if not inverse:
        return np.concatenate((np.real(point),np.imag(point)))
    else:
        return point[:len(point)//2] + 1j * point[len(point)//2:]
        
def CoordChange_C5L7(point, inverse=False):
    ## TO DO: vectorise
    if not inverse:
        one_idx = np.where(np.isclose(point,complex(1,0)))
        dropped_idx = pg._find_max_dQ_coords([point])
        c3_coords = np.delete(point,[one_idx,dropped_idx])
        return np.concatenate((np.real(c3_coords), np.imag(c3_coords), np.array([theta]))) ## theta will be point.theta
    else:
        z1 = 1+0j
        z2 = point[0] + point[3]*1j
        z3 = point[1] + point[4]*1j
        z4 = point[2] + point[5]*1j
        z5 = np.pow(-(1 + z2**5 + z3**5 + z4**5), 0.2) ## TO DO: generalise to cy_eqn(z1,z2,z3,z4)
        one_idx = point.one_idx ## make attribute
        dropped_idx = pg._find_max_dQ_coords([point]) ## make attribute as this point form wont be asosciated to pg  
        return np.insert(np.array([z1,z2,z3,z4]), [one_idx,dropped_idx], [z1,z5]) * np.exp(1j * point[-1])              


In [5]:
#Define the class for the link points
class LinkPoint:
    def __init__(self, cy_point):
        self.cy_point = cy_point
        self.one_idx = np.where(np.isclose(self.cy_point,complex(1,0)))[0][0] #...find the index which is set to 1 to define the patch
        self.dropped_idx = pg._find_max_dQ_coords([self.cy_point])[0]  #...find the index where the CY eqn is subbed, ## pg is external
        
        self.cy_point_in_S9 = self.cy_point/np.linalg.norm(self.cy_point) 
        self.theta = np.random.uniform(low=0., high=2*np.pi) #...sample a random angle
        self.link_point = self.cy_point_in_S9 * np.exp(1j * self.theta) #...multiple that angle in, the plane origin is the normalised cy point and orientation is implicitly set
        self.dtheta = self.cy_point_in_S9 * 1j * np.exp(1j * self.theta) #...the position vector 90degrees around the S1 is parallel to this d\theta, and cause its on sphere it is normalised already
        
        c3_coords = np.delete(self.link_point,[self.one_idx, self.dropped_idx]) #...produce the local complex CY coordinates
        self.link_point_local = np.concatenate((np.real(c3_coords), np.imag(c3_coords), np.array([self.theta]))) #...generate the link local coordinates
        
        self.holomorphic_volume_form = pg.holomorphic_volume_form(cy_point) ## pg is external
        #self.holomorphic_volume_form_C5 = ###

        self.cy_metric = cymetric_model.predict(tf.convert_to_tensor(CoordChange_C5R10(self.cy_point).reshape(1, 10)), verbose=0) 
        #self.kahler_form = ###

        #self.phi = wedge(self.kahler_form, self.dtheta) + np.imag(self.holomorphic_volume_form_C5) ###?

test_point = LinkPoint(points[0])